# Lie-ID-ASV on Colab
Run collection, training, and evaluation on GPU. Set your Drive path below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, sys, pathlib

# TODO: update this to the folder that contains asv_framework/ and the vendor repos
project_root = '/content/drive/MyDrive/Lie-ID-ASV'

os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print('cwd:', os.getcwd())

In [ ]:
!pip install -q torchdiffeq matplotlib numpy scipy

In [ ]:
import torch
print('GPU available:', torch.cuda.is_available(), 'CUDA version:', torch.version.cuda)
from asv_framework.models.se2_hamnode import SE2HamNODE

## Collect data (optional)
Generates zigzag data; skip if you already uploaded a dataset.

In [ ]:
!python -m asv_framework.collect_data \
  --episodes 50 \
  --steps 2000 \
  --dt 0.05 \
  --pattern zigzag \
  --period 150 \
  --fwd 0.7 \
  --diff 0.4 \
  --output asv_framework/data/otter_zigzag.npz

## Train
Warmup h=1 then h=10; priors keep masses/actuation in plausible ranges. Uses CUDA if available.

In [ ]:
!python -m asv_framework.training.train_otter_se2 \
  --data asv_framework/data/otter_zigzag.npz \
  --epochs 30 \
  --batch-size 512 \
  --lr 5e-5 \
  --hidden-dim 512 \
  --horizon 10 \
  --warmup-epochs 5 \
  --pos-scale 150.0 \
  --vel-scale 2.0 \
  --u-scale 50.0 \
  --max-deriv 150.0 \
  --act-prior-weight 1e-2 \
  --mass-prior-weight 1e-3 \
  --device cuda \
  --save-path asv_framework/data/otter_model.pt

## Evaluate
Open-loop rollout on zigzag using the trained model. Match the scales to training.

In [ ]:
!python -m asv_framework.training.eval_free_rollout \
  --model asv_framework/data/otter_model.pt \
  --steps 400 \
  --pattern zigzag \
  --period 150 \
  --fwd 0.7 \
  --diff 0.3 \
  --dt 0.05 \
  --hidden-dim 512 \
  --pos-scale 150.0 \
  --vel-scale 2.0 \
  --u-scale 50.0 \
  --output asv_framework/data/eval_plot.png